In [2]:
# Import the classes
from thesisv3.classism import MusicFileManager, MusicSegmentAnalyzer, MusicVisualizer

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\.School\Thesisv3\.venv\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [3]:
file_manager = MusicFileManager()
file_manager.display_selector()

In [ ]:
analyzer = MusicSegmentAnalyzer()
analyzer.load_score(file_manager.selected_file)
analyzer.analyze_segments()
analyzer.preprocess_segments()
# analyzer.calculate_distance_matrix()

In [6]:
visualizer = MusicVisualizer(analyzer)
visualizer.visualize_ir()

<music21.expressions.TextExpression 'Largo'>
<music21.layout.SystemLayout>
<music21.clef.TrebleClef>
<music21.tempo.MetronomeMark Quarter=50 (playback only)>
<music21.key.KeySignature of 2 flats>
<music21.meter.TimeSignature 4/4>
<music21.dynamics.Dynamic mp>
<music21.dynamics.Dynamic pp>
<music21.layout.SystemLayout>
<music21.layout.SystemLayout>
<music21.bar.Barline type=double>
<music21.expressions.TextExpression 'Moderato'>
<music21.tempo.MetronomeMark Quarter=110 (playback only)>
<music21.meter.TimeSignature 6/4>
<music21.dynamics.Dynamic mp>
<music21.layout.SystemLayout>
<music21.layout.SystemLayout>
<music21.layout.SystemLayout>
<music21.layout.SystemLayout>
<music21.tempo.MetronomeMark Quarter=125>
<music21.dynamics.Dynamic mf>
<music21.layout.SystemLayout>
<music21.layout.SystemLayout>
<music21.layout.SystemLayout>
<music21.expressions.TextExpression 'veloce'>
<music21.layout.PageLayout>
<music21.layout.SystemLayout>
<music21.tempo.MetronomeMark allegretto Quarter=110>
<music2

MusicXMLExportException: In part (Violín), measure (64): Cannot convert inexpressible durations to MusicXML.

In [ ]:
def visualize_notes_with_symbols(notes_with_symbols, original_score, all_parts=False):
    """
    Visualizes notes with their assigned IR symbols and colors in a music21 score.
    
    Parameters:
        notes_with_symbols (list): A list of tuples containing (element, symbol, color).
        original_score (music21.stream.Score): The original score used as template.
    
    Returns:
        music21.stream.Score: A new score with annotated symbols and colors.
    """
    # Make a deep copy of the original score to preserve its structure
    new_score = copy.deepcopy(original_score)
    parts_to_process = new_score.parts if hasattr(new_score, 'parts') else [new_score]
    if not all_parts and hasattr(new_score, 'parts'):
        parts_to_process = [new_score.parts[0]]
    
    # Create an iterator over the provided symbols list
    symbols_iter = iter(notes_with_symbols)
    
    for part in parts_to_process:
        for measure in part.getElementsByClass(stream.Measure):
            # Process each element in the measure
            for element in measure:
                # If the element is a Voice, process its subelements.
                if isinstance(element, stream.Voice):
                    # if (element.id is not None and element.id != '1'):
                    #     continue
                    for subelement in element.flatten().getElementsByClass([note.Note, note.Rest, chord.Chord]):
                        try:
                            symbol_element, symbol, color, _ = next(symbols_iter)
                            # Apply color and add lyric if the elements match.
                            if subelement == symbol_element:
                                subelement.style.color = color
                                subelement.lyric = symbol
                            else:
                                print(f"Expected: {symbol_element}, Found: {element}")
                        except StopIteration:
                            print(f"stopping {element}")
                            break  # No more symbols to assign.
                # Process standalone elements (notes, rests, chords).
                elif isinstance(element, (note.Note, note.Rest, chord.Chord)):
                    try:
                        symbol_element, symbol, color, _ = next(symbols_iter)
                        if element == symbol_element:
                            element.style.color = color
                            element.lyric = symbol
                        else:
                            print(f"Expected: {symbol_element}, Found: {element}")
                    except StopIteration:
                        print(f"stopping {element}")
                        break
                else:
                    print(element)
    return new_score

In [ ]:
analyzer.nmat

In [ ]:
import copy
from music21 import *
visualize_notes_with_symbols(analyzer.ir_symbols, analyzer.parsed_score).show()

In [ ]:
visualizer.visualize_multiple_segments()

In [ ]:
visualizer.visualize_colored_segments()

In [ ]:
visualizer.visualize_knn_graph(k=4, title="Segment Analysis")

In [ ]:
def calculate_ir_symbol_old(interval1, interval2, threshold=5):
    """
    Calculates the IR (Intervallic Relationship) symbol based on the intervals between notes.

    Parameters:
    interval1 (int): The interval between the first and second notes.
    interval2 (int): The interval between the second and third notes.
    threshold (int): The threshold value for determining the type of relationship.

    Returns:
    str: The IR symbol representing the relationship between the intervals.
    """
    direction = interval1 * interval2
    abs_difference = abs(interval2 - interval1)

    if direction > 0 and abs_difference < threshold:
        return 'P'  # Process
    elif interval1 == interval2 == 0:
        return 'D'  # Duplication
    elif (interval1 * interval2 < 0) and (-threshold <= abs(abs_difference) <= threshold) and (abs(interval2) != abs(interval1)):
        return 'IP'  # Intervallic Process
    elif (interval1 * interval2 < 0) and (abs(interval2) == abs(interval1)):
        return 'ID'  # Intervallic Duplication
    elif (interval1 * interval2 > 0) and (abs_difference >= threshold) and (abs(interval1) <= threshold):
        return 'VP'  # Vector Process
    elif (interval1 * interval2 < 0) and (abs(abs_difference) >= threshold) and (abs(interval1) >= threshold):
        return 'R'  # Reversal
    elif (interval1 * interval2 > 0) and (abs(abs_difference) >= threshold) and (abs(interval1) >= threshold):
        return 'IR'  # Intervallic Reversal
    elif (interval1 * interval2 < 0) and (abs_difference >= threshold) and (abs(interval1) <= threshold):
        return 'VR'  # Vector Reversal
    elif interval2 == 0 and not (interval1 < -5 or interval1 > 5):
        return 'IP'
    elif interval2 == 0 and (interval1 < -5 or interval1 > 5):
        return 'R'
    elif interval1 == 0 and not (interval2 < -5 or interval2 > 5):
        return 'P'
    elif interval1 == 0 and (interval2 < -5 or interval2 > 5):
        return 'VR'


In [ ]:
from music21 import chord, note
from thesisv3.preprocessing.preprocessing import calculate_ir_symbol
from thesisv3.analysis.visualization import visualize_notes_with_symbols

def assign_ir_symbols_old(note_array):
    """
    Assigns IR symbols and colors to each element in the score array.

    Parameters:
    score_array (list): A list of music21 note and chord elements.

    Returns:
    list: A list of tuples containing each element, its IR symbol, and its color.
    """
    symbols = []
    current_group = []
    group_pitches = []

    color_map = {
        'P': 'blue',  # IR1: P (Process)
        'D': 'green',  # IR2: D (Duplication)
        'IP': 'red',  # IR3: IP (Intervallic Process)
        'ID': 'orange',  # IR4: ID (Intervallic Duplication)
        'VP': 'purple',  # IR5: VP (Vector Process)
        'R': 'cyan',  # IR6: R (Reversal)
        'IR': 'magenta',  # IR7: IR (Intervallic Reversal)
        'VR': 'yellow',  # IR8: VR (Vector Reversal)
        'M': 'pink',  # IR9: M (Monad)
        'd': 'lime',  # IR10 d (Dyad)
    }

    def evaluate_current_group():
        if len(current_group) == 3:
            interval1 = group_pitches[1] - group_pitches[0]
            interval2 = group_pitches[2] - group_pitches[1]
            symbol = calculate_ir_symbol_old(interval1, interval2)
            color = color_map.get(symbol, 'black')  # Default to black if symbol is not predefined
            symbols.extend([(note, symbol, color) for note in current_group])
        elif len(current_group) == 2:
            symbols.extend([(note, 'd', color_map['d']) for note in current_group])  # Dyad
        elif len(current_group) == 1:
            symbols.extend([(note, 'M', color_map['M']) for note in current_group])  # Monad
        current_group.clear()
        group_pitches.clear()

    for element in note_array:
        if isinstance(element, note.Note):
            current_group.append(element)
            group_pitches.append(element.pitch.ps)
            if len(current_group) == 3:
                evaluate_current_group()
        elif isinstance(element, chord.Chord):
            current_group.append(element)
            group_pitches.append(element.root().ps)
            if len(current_group) == 3:
                evaluate_current_group()
        elif isinstance(element, note.Rest):
            rest_tuple = (element, 'rest', 'black')
            evaluate_current_group()
            symbols.append(rest_tuple)
        else:
            if current_group:
                evaluate_current_group()

    # Handle any remaining notes
    if current_group:
        evaluate_current_group()

    return symbols

In [ ]:
from music21 import stream

def visualize_notes_with_symbols_old(notes_with_symbols):
    """
    Visualizes notes with their assigned IR symbols and colors in a music21 score.

    Parameters:
    notes_with_symbols (list): A list of tuples containing each note, its IR symbol, and its color.

    Returns:
    None
    """
    s = stream.Score()
    part = stream.Part()
    for note, symbol, color in notes_with_symbols:
        note.style.color = color
        note.lyric = symbol
        part.append(note)
    s.append(part)
    s.show()

In [ ]:
def assign_ir_symbols_new(note_array):
    """
    Assigns IR symbols and colors to each element in the score array.

    Parameters:
    score_array (list): A list of music21 note and chord elements.

    Returns:
    list: A list of tuples containing each element, its IR symbol, and its color.
    """
    symbols = []
    current_group = []
    group_pitches = []
    last_beam_status = None

    color_map = {
        'P': 'blue',  # IR1: P (Process)
        'D': 'green',  # IR2: D (Duplication)
        'IP': 'red',  # IR3: IP (Intervallic Process)
        'ID': 'orange',  # IR4: ID (Intervallic Duplication)
        'VP': 'purple',  # IR5: VP (Vector Process)
        'R': 'cyan',  # IR6: R (Reversal)
        'IR': 'magenta',  # IR7: IR (Intervallic Reversal)
        'VR': 'yellow',  # IR8: VR (Vector Reversal)
        'M': 'pink',  # IR9: M (Monad)
        'd': 'lime',  # IR10 d (Dyad)
    }

    def evaluate_current_group():
        if len(current_group) >= 3:
            interval1 = group_pitches[1] - group_pitches[0]
            interval2 = group_pitches[2] - group_pitches[1]
            symbol = calculate_ir_symbol(interval1, interval2)
            color = color_map.get(symbol, 'black')  # Default to black if symbol is not predefined
            symbols.extend([(note, symbol, color) for note in current_group])
        elif len(current_group) == 2:
            symbols.extend([(note, 'd', color_map['d']) for note in current_group])  # Dyad
        elif len(current_group) == 1:
            symbols.extend([(note, 'M', color_map['M']) for note in current_group])  # Monad
        current_group.clear()
        group_pitches.clear()

    def get_beam_status(e):
        if not isinstance(e, (note.Note, chord.Chord)):
            return None
        beam_status = 'single'  # Default status for unbeamed notes
        if e.beams:
            # Get beam types
            beam_types = [beam.type for beam in e.beams.beamsList]
            if 'start' in beam_types:
                beam_status = 'start'
            elif 'continue' in beam_types:
                beam_status = 'continue'
            elif 'stop' in beam_types:
                beam_status = 'stop'
            elif 'partial' in beam_types:
                beam_status = 'partial'
        return beam_status

    num_notes = len(note_array)
    i = 0
    while i < num_notes:
        element = note_array[i]
        if isinstance(element, (note.Note, chord.Chord)):
            # Get beam status
            beam_status = get_beam_status(element)
            current_group.append(element)
            if isinstance(element, note.Note):
                group_pitches.append(element.pitch.ps)
            elif isinstance(element, chord.Chord):
                group_pitches.append(element.root().ps)

            # BEAM
            if (last_beam_status in ['start', 'continue', 'partial']) and (
                    beam_status in ['continue', 'stop', 'partial']):
                if beam_status == 'stop':

                    # TODO: REVISIT IF IT IS WEIRD
                    if len(current_group) == 2:
                        i += 1
                        element2 = note_array[i]
                        if isinstance(element2, (note.Note, chord.Chord)):
                            # Get beam status
                            beam_status = get_beam_status(element2)
                            current_group.append(element2)
                            if isinstance(element2, note.Note):
                                group_pitches.append(element2.pitch.ps)
                            elif isinstance(element2, chord.Chord):
                                group_pitches.append(element2.root().ps)
                        elif isinstance(element, note.Rest):
                            rest_tuple = (element, 'rest', 'black')
                            evaluate_current_group()
                            symbols.append(rest_tuple)
                        else:
                            if current_group:
                                evaluate_current_group()
                    last_beam_status = beam_status

            elif i < num_notes - 1 and get_beam_status(note_array[i + 1]) == 'start' and beam_status == 'single':
                evaluate_current_group()
            if len(current_group) == 3:
                evaluate_current_group()
            last_beam_status = beam_status

        elif isinstance(element, note.Rest):
            rest_tuple = (element, 'rest', 'black')
            evaluate_current_group()
            symbols.append(rest_tuple)
            last_beam_status = None
            # symbols.append("POOP")
        else:
            if current_group:
                evaluate_current_group()
            last_beam_status = None
        i += 1

    # Handle any remaining notes
    if current_group:
        evaluate_current_group()

    return symbols


In [ ]:
narr = analyzer.narr
score = analyzer.parsed_score

In [ ]:
symbols = assign_ir_symbols_new(narr)
visualize_notes_with_symbols(symbols, score).show()

In [ ]:
import pandas as pd


def parse_score_elements(score: stream.Score, all_parts: bool = False) -> tuple[pd.DataFrame, list, list]:
    """
    Parses a music21 score object into a DataFrame of note attributes and lists of note and chord elements.
    By default, only processes the first part unless all_parts=True.

    Parameters:
    score (music21.stream.Score): The music21 score object to parse.
    all_parts (bool): If True, process all parts. If False, only process the first part. Defaults to False.

    Returns:
    tuple: A tuple containing:
        - pd.DataFrame: A DataFrame with onset (global and relative to measure), duration, MIDI pitch, pitch class, octave, and beat strength for each note.
        - list: A list of note and chord elements.
        - list: A list of all elements processed.
    """
    trashed_elements = 0
    narr = []
    sarr = []
    nmat = pd.DataFrame(columns=[
        'onset_beats',  # Onset in beats for the whole piece
        'onset_beats_in_measure',  # Onset relative to the measure
        'duration_beats',
        'midi_pitch',
        'pitch_class',
        'octave',
        'beat_strength'
    ])

    onset_beat = 0
    parts_to_process = score.parts[0]

    for part in parts_to_process:
        for measure in part.getElementsByClass(stream.Measure):
            measure_offset_in_score = measure.getOffsetInHierarchy(score)
            for element in measure:
                sarr.append(element)
                # Onset relative to the measure
                onset_beat_in_measure = element.offset
                # Onset relative to the whole piece
                duration_beats = element.duration.quarterLength

                # Beat strength calculation
                beat_strength = element.beatStrength if hasattr(element, 'beatStrength') else None

                if isinstance(element, chord.Chord):
                    # Use the root note of the chord
                    root_note = element.root()
                    pitch_class = root_note.pitchClass
                    octave = root_note.octave
                    midi_pitch = root_note.midi
                    row = [
                        onset_beat,
                        onset_beat_in_measure,
                        duration_beats,
                        midi_pitch,
                        pitch_class,
                        octave,
                        beat_strength
                    ]
                    nmat.loc[len(nmat)] = row
                    narr.append(element)
                elif isinstance(element, note.Rest):
                    # Represent rest with None for pitch attributes
                    row = [
                        onset_beat,
                        onset_beat_in_measure,
                        duration_beats,
                        0,
                        0,
                        0,
                        beat_strength
                    ]
                    nmat.loc[len(nmat)] = row
                    narr.append(element)
                elif isinstance(element, note.Note):
                    pitch_class = element.pitch.pitchClass
                    octave = element.pitch.octave
                    midi_pitch = element.pitch.midi
                    row = [
                        onset_beat,
                        onset_beat_in_measure,
                        duration_beats,
                        midi_pitch,
                        pitch_class,
                        octave,
                        beat_strength
                    ]
                    nmat.loc[len(nmat)] = row
                    narr.append(element)
                else:
                    trashed_elements += 1
                onset_beat += duration_beats

    return nmat, narr, sarr


In [ ]:
import pandas as pd
trashed_elements = 0
narr = []
sarr = []
nmat = pd.DataFrame(columns=[
    'onset_beats',  # Onset in beats for the whole piece
    'onset_beats_in_measure',  # Onset relative to the measure
    'duration_beats',
    'midi_pitch',
    'pitch_class',
    'octave',
    'beat_strength'
])

onset_beat = 0
parts_to_process = [score.parts[0]]

In [ ]:
for part in parts_to_process:
    for measure in part.getElementsByClass(stream.Measure):
        for element in measure:
            print(element)
            # if isinstance(element, stream.Voice):
            #     print("\n")
            #     print(element)
            #     duration_beats = element.duration.quarterLength
            #     print(duration_beats)
            #     print(element.offset)
            #     for subelement in element.flatten():
            #         print(subelement)
            #         duration_beats = subelement.duration.quarterLength
            #         print(duration_beats)
            #         print(subelement.offset)
            # else:
            #     print(element)

In [ ]:
for part in parts_to_process:
    for measure in part.getElementsByClass(stream.Measure):
        measure_offset_in_score = measure.getOffsetInHierarchy(score)
        for element in measure:
            sarr.append(element)
            # Onset relative to the measure
            onset_beat_in_measure = element.offset
            # Onset relative to the whole piece
            duration_beats = element.duration.quarterLength

            # Beat strength calculation
            beat_strength = element.beatStrength if hasattr(element, 'beatStrength') else None

            if isinstance(element, chord.Chord):
                # Use the root note of the chord
                root_note = element.root()
                pitch_class = root_note.pitchClass
                octave = root_note.octave
                midi_pitch = root_note.midi
                row = [
                    onset_beat,
                    onset_beat_in_measure,
                    duration_beats,
                    midi_pitch,
                    pitch_class,
                    octave,
                    beat_strength
                ]
                nmat.loc[len(nmat)] = row
                narr.append(element)
            elif isinstance(element, note.Rest):
                # Represent rest with None for pitch attributes
                row = [
                    onset_beat,
                    onset_beat_in_measure,
                    duration_beats,
                    0,
                    0,
                    0,
                    beat_strength
                ]
                nmat.loc[len(nmat)] = row
                narr.append(element)
            elif isinstance(element, note.Note):
                pitch_class = element.pitch.pitchClass
                octave = element.pitch.octave
                midi_pitch = element.pitch.midi
                row = [
                    onset_beat,
                    onset_beat_in_measure,
                    duration_beats,
                    midi_pitch,
                    pitch_class,
                    octave,
                    beat_strength
                ]
                nmat.loc[len(nmat)] = row
                narr.append(element)
            else:
                trashed_elements += 1
            onset_beat += duration_beats

In [ ]:
narr

gippity


In [ ]:

trashed_elements = 0
narr = []
sarr = []
nmat = pd.DataFrame(columns=[
    'onset_beats',
    'onset_beats_in_measure',
    'duration_beats',
    'midi_pitch',
    'pitch_class',
    'octave',
    'beat_strength'
])

for part in score.parts:
    for element in part.flatten():
        sarr.append(element)
        print(element)

In [ ]:

        global_onset = element.offset
        # Compute onset relative to the current measure if available.
        measure_context = element.getContextByClass('Measure')
        if measure_context is not None:
            onset_in_measure = element.offset - measure_context.offset
        else:
            onset_in_measure = element.offset

        duration_beats = element.duration.quarterLength
        beat_strength = getattr(element, 'beatStrength', None)

        if isinstance(element, chord.Chord):
            root = element.root()
            midi_pitch = root.midi
            pitch_class = root.pitchClass
            octave = root.octave
            row = [
                global_onset,
                onset_in_measure,
                duration_beats,
                midi_pitch,
                pitch_class,
                octave,
                beat_strength
            ]
            nmat.loc[len(nmat)] = row
            narr.append(element)
        elif isinstance(element, note.Note):
            midi_pitch = element.pitch.midi
            pitch_class = element.pitch.pitchClass
            octave = element.pitch.octave
            row = [
                global_onset,
                onset_in_measure,
                duration_beats,
                midi_pitch,
                pitch_class,
                octave,
                beat_strength
            ]
            nmat.loc[len(nmat)] = row
            narr.append(element)
        elif isinstance(element, note.Rest):
            # For rests, we use None for pitch-related attributes.
            row = [
                global_onset,
                onset_in_measure,
                duration_beats,
                None,
                None,
                None,
                beat_strength
            ]
            nmat.loc[len(nmat)] = row
            narr.append(element)
        else:
            trashed_elements += 1



In [ ]:
sarr

In [ ]:
import copy
from music21 import stream, note, chord

def color_stream(original_score):
    """
    Creates a deep copy of the input score and colors all elements inside Voice objects.

    Parameters:
    original_score (music21.stream.Score): The original music21 score.

    Returns:
    music21.stream.Score: A new score with colored Voice elements.
    """
    # Make a deep copy of the original score to preserve its structure
    new_score = copy.deepcopy(original_score)

    # Iterate over the parts of the new_score
    for part in new_score.parts:
        # Iterate over measures in the part
        for measure in part.getElementsByClass(stream.Measure):
            # Iterate over elements in the measure
            for element in measure:
                if isinstance(element, stream.Voice):  # Check if the element is a Voice
                    for sub_element in element:  # Iterate through the Voice's elements
                        if isinstance(sub_element, (note.Note, chord.Chord, note.Rest)):
                            sub_element.style.color = "blue"  # Set color for notes/chords/rests
                elif isinstance(element, (note.Note, chord.Chord, note.Rest)):
                    element.style.color = "black"  # Keep regular elements uncolored or default

    return new_score


In [ ]:
color_stream(score).show()

In [ ]:
from fractions import Fraction
import copy
import copy
import os
import tempfile
from fractions import Fraction

import ipywidgets as widgets
import pandas as pd
from IPython.display import display, clear_output
from music21 import (
    stream,
    note,
    chord,
    expressions,
    layout,
    environment
)
from pygame import mixer


def visualize_segment(segments, segment_index, original_score, show_score=True):
    """
    Visualizes a specific segment from a musical score using an additive approach,
    building a new score with only the desired elements.
    """
    def approximate_to_fraction(df, column='onset_beats', max_denominator=16):
        df = df.copy()
        df[column] = df[column].apply(
            lambda x: float(Fraction(x).limit_denominator(max_denominator))
        )
        return df

    # Define the color map for IR symbols
    color_map = {
        'P': 'blue',  # IR1: P (Process)
        'D': 'green',  # IR2: D (Duplication)
        'IP': 'red',  # IR3: IP (Intervallic Process)
        'ID': 'orange',  # IR4: ID (Intervallic Duplication)
        'VP': 'purple',  # IR5: VP (Vector Process)
        'R': 'cyan',  # IR6: R (Reversal)
        'IR': 'magenta',  # IR7: IR (Intervallic Reversal)
        'VR': 'yellow',  # IR8: VR (Vector Reversal)
        'M': 'pink',  # IR9: M (Monad)
        'd': 'lime',  # IR10 d (Dyad)
        'rest': 'black'  # Default color for rests
    }

    if not isinstance(segments, list) or segment_index >= len(segments):
        raise ValueError(f"Invalid segment index. Must be between 0 and {len(segments) - 1}")

    # Make a deep copy of the original score to preserve its structure
    score = copy.deepcopy(original_score)

    # Create a completely new score
    new_score = stream.Score()

    # First check if we have any staff groups (like piano's grand staff)
    part_groups = []
    for element in score.elements:
        if isinstance(element, layout.StaffGroup):
            part_groups.append(copy.deepcopy(element))

    # Add any staff groups to the new score first
    for group in part_groups:
        new_score.append(group)

    # Copy parts while preserving their relationships
    for part in score.parts:
        new_part = stream.Part()

        # Preserve staff layout information
        for elem in part.getElementsByClass('LayoutClass'):
            new_part.append(copy.deepcopy(elem))

        # Copy measures and their contents
        for measure in part.getElementsByClass('Measure'):
            new_measure = stream.Measure()
            new_measure.number = measure.number

            # Copy time signature if it exists
            ts = measure.timeSignature
            if ts:
                new_measure.timeSignature = copy.deepcopy(ts)
            # Copy clef if it exists
            cl = measure.clef
            if cl:
                new_measure.clef = copy.deepcopy(cl)
            # Copy key signature if it exists
            ks = measure.keySignature
            if ks:
                new_measure.keySignature = copy.deepcopy(ks)

            # Copy notes, rests, and chords
            for elem in measure.getElementsByClass(['Note', 'Rest', 'Chord']):
                new_measure.append(copy.deepcopy(elem))

            new_part.append(new_measure)

        # Preserve any staff identifiers or group memberships
        if hasattr(part, 'staffName'):
            new_part.staffName = part.staffName
        if hasattr(part, 'groupName'):
            new_part.groupName = part.groupName

        new_score.append(new_part)

    # Add segment number as a text expression at the top
    segment_text = expressions.TextExpression(f'Segment {segment_index}')
    segment_text.style.absoluteY = 75  # Position above the staff
    segment_text.style.fontSize = 14
    first_measure = new_score.parts[0].getElementsByClass('Measure')[0]
    first_measure.insert(0, segment_text)

    # Get the selected segment
    segment = segments[segment_index]
    segment = approximate_to_fraction(segment, column='onset_beats', max_denominator=16)

    # Create a set of onsets in the segment for quick lookup
    segment_onsets = set(segment['onset_beats'].values)

    # Iterate over the parts of the new_score
    for part in new_score.parts:
        measures_to_keep = []
        # First pass: identify measures with segment notes and transform notes
        for measure in part.getElementsByClass(stream.Measure):
            has_segment_notes = False
            for element in measure:
                if isinstance(element, (note.Note, note.Rest, chord.Chord)):
                    element_offset = element.getOffsetInHierarchy(new_score)

                    if element_offset in segment_onsets:
                        has_segment_notes = True
                        # Find the corresponding row in our segment DataFrame
                        segment_row = segment[segment['onset_beats'] == element_offset].iloc[0]

                        # If it's a rest in our segment data
                        if segment_row['midi_pitch'] == 0:
                            new_element = note.Rest(quarterLength=element.duration.quarterLength)
                            new_element.style.color = color_map['rest']
                        else:
                            # Create a new note with the pitch from our segment
                            new_element = note.Note(
                                pitch=segment_row['midi_pitch'],
                                quarterLength=element.duration.quarterLength
                            )

                            # Add IR symbol and color if available
                            if 'ir_symbol' in segment_row:
                                ir_symbol = segment_row['ir_symbol']
                                new_element.lyric = ir_symbol
                                if ir_symbol in color_map:
                                    new_element.style.color = color_map[ir_symbol]

                        # Replace the element in the measure
                        measure.replace(element, new_element)
                    else:
                        # Remove notes not in our segment
                        measure.remove(element)

            if has_segment_notes:
                measures_to_keep.append(measure)

        # Clear all measures from the part
        for measure in part.getElementsByClass(stream.Measure):
            part.remove(measure)

        # Add back only the measures we want to keep
        for measure in measures_to_keep:
            part.append(measure)

        # If the part is empty after removing measures, remove it from the score
        if len(part.getElementsByClass(stream.Measure)) == 0:
            new_score.remove(part)

    # --- FIX: Ensure the score is well-formed before showing/exporting ---
    new_score.makeNotation(inPlace=True)

    # Show the updated score if requested
    if show_score:
        new_score.show()

    return new_score


In [ ]:
analyzer.segments[2]

In [ ]:
visualize_segment(analyzer.segments, 3, analyzer.parsed_score)

In [ ]:
visualize_notes_with_symbols(analyzer.ir_symbols, analyzer.parsed_score).show()
